# Biblioteca de Algoritmos - Lab 03

Nos últimos anos, muitas bibliotecas RL foram desenvolvidas. Essas bibliotecas foram projetadas para ter todas as ferramentas necessárias para implementar e testar agentes de Aprendizado por Reforço .

Ainda assim, elas se diferem muito. É por isso que é importante escolher uma biblioteca que seja rápida, confiável e relevante para sua tarefa de RL. Do ponto de vista técnico, existem algumas coisas a se ter em mente ao considerar uma bilioteca para RL.

- **Suporte para bibliotecas de aprendizado de máquina existentes:** Como o RL normalmente usa algoritmos baseados em gradiente para aprender e ajustar funções de política, você vai querer que ele suporte sua biblioteca favorita (Tensorflow, Keras, Pytorch, etc.)
- **Escalabilidade:** RL é computacionalmente intensivo e ter a opção de executar de forma distribuída torna-se importante ao atacar ambientes complexos.
- **Composibilidade:** Os algoritmos de RL normalmente envolvem simulações e muitos outros componentes. Você vai querer uma biblioteca que permita reutilizar componentes de algoritmos de RL, que seja compatível com várias estruturas de aprendizado profundo.

[Aqui](https://docs.google.com/spreadsheets/d/1ZWhViAwCpRqupA5E_xFHSaBaaBZ1wAjO6PvmmEEpXGI/edit#gid=0) você consegue visualizar uma lista com algumas bibliotecas existentes.

<img src="https://i1.wp.com/neptune.ai/wp-content/uploads/RL-tools.png?resize=1024%2C372&ssl=1" width=500>


## Ray RLlib

[Ray](https://docs.ray.io/en/latest/) é uma plataforma de execução distribuída que fornece bases para paralelismo e escalabilidade que são simples de usar e permitem que os programas Python sejam escalados em qualquer lugar, de um notebook a um grande cluster. Além disso, construída sobre o Ray, temos a [RLlib](https://docs.ray.io/en/latest/rllib.html), que fornece uma API unificada que pode ser aproveitada em uma ampla gama de aplicações.

<br>

<img src="https://miro.medium.com/max/1838/1*_bomm09XtiZfQ52Kfz9Ciw.png" width=600>


A RLlib foi projetada para oferecer suporte a várias estruturas de aprendizado profundo (TensorFlow e PyTorch) e pode ser acessada por meio de uma API Python simples. Atualmente, ela vem com uma [série de algoritmos RL](https://docs.ray.io/en/latest/rllib-algorithms.html#available-algorithms-overview).

Em particular, a RLlib permite um desenvolvimento rápido porque torna mais fácil construir algoritmos RL escaláveis ​​por meio da reutilização e montagem de implementações existentes. A RLlib também permite que os desenvolvedores usem redes neurais criadas com várias estruturas de aprendizado profundo e se integra facilmente a simuladores de terceiros.


## Configuração

Você precisará fazer uma cópia deste notebook em seu Google Drive antes de editar. Você pode fazer isso com **Arquivo → Salvar uma cópia no Drive**.

In [1]:
import os
#from google.colab import drive
#drive.mount("/content/gdrive")

In [2]:
# Seu trabalho será armazenado em uma pasta chamada `minicurso_rl` por padrão 
# para evitar que o tempo limite da instância do Colab exclua suas edições

DRIVE_PATH = "../minicurso_rl/lab03"
#DRIVE_PYTHON_PATH = DRIVE_PATH.replace("\\", "")
if not os.path.exists(DRIVE_PATH):
  %mkdir -p $DRIVE_PATH

In [3]:
# Ambiente da competição
#!pip install --upgrade ceia-soccer-twos > /dev/null 2>&1
# a versão do ray compatível com a implementação dos agentes disponibilizada é a 1.4.0
#!pip install 'aioredis==1.3.1' > /dev/null 2>&1 
#!pip install 'aiohttp==3.7.4' > /dev/null 2>&1 
#!pip install 'ray==1.4.0' > /dev/null 2>&1 
#!pip install 'ray[rllib]==1.4.0' > /dev/null 2>&1 
#!pip install 'ray[tune]==1.4.0' > /dev/null 2>&1 
#!pip install torch > /dev/null 2>&1 
#!pip install lz4 > /dev/null 2>&1 

# Dependências necessárias para gravar os vídeos
#!apt-get install -y xvfb x11-utils > /dev/null 2>&1 
#!pip install pyvirtualdisplay==0.2.* > /dev/null 2>&1 

In [4]:
#! wget http://www.atarimania.com/roms/Roms.rar
#! mkdir ../content/ROM/
#! unrar e ../content/Roms.rar ../content/ROM/ -y
#! python -m atari_py.import_roms ../content/ROM/ > /dev/null 2>&1

In [5]:
# Inicializa uma instância de um display virtual
from pyvirtualdisplay import Display
display = Display(visible=False, size=(1400, 900))
_ = display.start()

In [6]:
# Carrega a extensão do notebook TensorBoard
%load_ext tensorboard

## Ambiente

O OpenAI Gym possui um wrapper VideoRecorder que pode gravar um vídeo do ambiente em formato MP4. Abaixo iremos interagir no ambiente do [Carpole](https://gym.openai.com/envs/CartPole-v0/) executando ações aleatórias e gravar o resultado.

In [88]:
import gym
from gym.wrappers.monitoring.video_recorder import VideoRecorder

environment_id = "CartPole-v0"

In [8]:
import gym
from gym.wrappers.monitoring.video_recorder import VideoRecorder

env = gym.make(environment_id)
before_training = os.path.join(
    DRIVE_PATH, "{}_before_training.mp4".format(environment_id)
)

In [9]:
print(before_training)

video = VideoRecorder(env, before_training)
env.reset()
for i in range(200):
  env.render()
  video.capture_frame()
  observation, reward, done, info = env.step(env.action_space.sample())

video.close()
env.close()

../minicurso_rl/lab03/CartPole-v0_before_training.mp4


/home/eduardo/anaconda3/envs/rl/lib/python3.7/site-packages/gym/logger.py:34: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


O código acima salvou o arquivo de vídeo no seu Drive. Para exibi-lo no notebook, você precisa de uma função auxiliar.

In [10]:
from base64 import b64encode
def render_mp4(videopath: str) -> str:
  mp4 = open(videopath, 'rb').read()
  base64_encoded_mp4 = b64encode(mp4).decode()
  return f'<video width=400 controls><source src="data:video/mp4;' \
         f'base64,{base64_encoded_mp4}" type="video/mp4"></video>'

O código abaixo renderiza os resultados. Você deve obter um vídeo semelhante ao abaixo.

In [11]:
from IPython.display import HTML
html = render_mp4(before_training)
HTML(html)

## Treinando um agente de Aprendizado por Reforço

Primeiro, vamos começar a executar o Ray em segundo plano. Executar um `ray.shutdown()` seguido por um `ray.init()` deve dar início às coisas.

In [12]:
import ray

ray.shutdown()
ray.init(ignore_reinit_error=True, include_dashboard=False)

{'node_ip_address': '192.168.130.2',
 'raylet_ip_address': '192.168.130.2',
 'redis_address': '192.168.130.2:6379',
 'object_store_address': '/tmp/ray/session_2021-11-12_23-54-36_935172_22643/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-11-12_23-54-36_935172_22643/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2021-11-12_23-54-36_935172_22643',
 'metrics_export_port': 62556,
 'node_id': '325cbe677fea6c1547686fe403f55f09f5b14875188dc3525b6b8915'}

### Basic Python API

Em alto nível, RLlib fornece uma classe Trainer que contém uma política para interação com o ambiente. Por meio da interface do Trainer, a política pode ser treinada, avaliada ou computar uma ação. 

Para cada algoritmo gostaríamos de configurar os parâmetros (taxa de aprendizado, tamanho da rede, tamanho do batch, etc.) de acordo com a nossa aplicação.  Para isso o Ray fornece dois níveis de paramêtros que podemos alterar. Primeiramente temos os parâmetros comuns a todos os algoritmos. Você pode conferir uma lista com os parâmetros disponíveis através desse [link](https://docs.ray.io/en/latest/rllib-training.html#common-parameters).

E para cada [algoritmo disponível no ray](https://docs.ray.io/en/latest/rllib-algorithms.html#available-algorithms-overview) temos os parâmetros específicos. Na imagem abaixo podemos ver os parâmetros específicos para o algoritmo [Policy Gradient](https://docs.ray.io/en/latest/rllib-algorithms.html#policy-gradients).


<img src='https://drive.google.com/uc?id=1yKJDJViHE_F9JH7NTQMYtQL3KLBJoJyk' width="500" >


In [ ]:
import ray
import ray.rllib.agents.pg as pg
from ray.tune.logger import pretty_print

config = pg.DEFAULT_CONFIG.copy()
config["num_gpus"] = 0
config["num_workers"] = 1
config["lr"] = 0.0004
config["framework"] = "torch"

trainer = pg.PGTrainer(config=config, env=environment_id)
episodes = 1000

for i in range(episodes):
   # Executa uma iteração de treinamento da política com Policy Gradient (PG)
   result = trainer.train()
   print(pretty_print(result))

   if i % 100 == 0:
       checkpoint = trainer.save()
       print("checkpoint saved at", checkpoint)

last_checkpoint = trainer.save()

In [14]:
print("Last checkpoint saved at", last_checkpoint)

Last checkpoint saved at /home/eduardo/ray_results/PG_CartPole-v0_2021-11-12_23-54-41x6aj24su/checkpoint_001000/checkpoint-1000


Agora vamos criar outro vídeo, mas desta vez escolha a ação recomendada pelo modelo treinado em vez de agir aleatoriamente.

In [15]:
trainer = pg.PGTrainer(config=config, env=environment_id)
trainer.restore(last_checkpoint)

after_training = os.path.join(
    DRIVE_PATH, "{}after_training_basic_api.mp4".format(environment_id)
)
after_video = VideoRecorder(env, after_training)
observation = env.reset()
done = False
while not done:
  env.render()
  after_video.capture_frame()
  action = trainer.compute_action(observation)
  observation, reward, done, info = env.step(action)
after_video.close()
env.close()
html = render_mp4(after_training)
HTML(html)

2021-11-12 23:57:15,013	WARNING util.py:53 -- Install gputil for GPU system monitoring.
2021-11-12 23:57:15,018	INFO trainable.py:378 -- Restored on 192.168.130.2 from checkpoint: /home/eduardo/ray_results/PG_CartPole-v0_2021-11-12_23-54-41x6aj24su/checkpoint_001000/checkpoint-1000
2021-11-12 23:57:15,018	INFO trainable.py:385 -- Current state after restoring: {'_iteration': 1000, '_timesteps_total': None, '_time_total': 140.12676167488098, '_episodes_total': 1207}


### Usando ambiente ou modelos personalizados

A API Python fornece a flexibilidade necessária para aplicar o RLlib a novos problemas. Você precisará usar esta API se desejar usar ambientes ou modelos personalizados com RLlib. Abaixo veremos um exemplo de um ambiente e um modelo customizado.

<br>


Para maiores informações veja em [APIs Python avançadas](https://docs.ray.io/en/latest/rllib-training.html#advanced-python-apis).

In [16]:
import gym
from gym.spaces import Discrete, Box
import numpy as np
import os
import random

import torch
import torch.nn as nn

import ray
from ray import tune
from ray.rllib.agents import pg
from ray.rllib.env.env_context import EnvContext
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.models.torch.fcnet import FullyConnectedNetwork as TorchFC
from ray.tune.logger import pretty_print

In [17]:
class SimpleCorridor(gym.Env):
    """Exemplo de um ambiente personalizado em que você tem que andar por um 
    corredor. Você pode configurar o comprimento do corredor através da 
    configuração do ambiente."""

    def __init__(self, config: EnvContext):
        self.end_pos = config["corridor_length"]
        self.cur_pos = 0
        self.action_space = Discrete(2)
        self.observation_space = Box(
            0.0, self.end_pos, shape=(1, ), dtype=np.float32)
        # Define a seed. É usado apenas para a recompensa final.
        self.seed(config.worker_index * config.num_workers)

    def reset(self):
        self.cur_pos = 0
        return [self.cur_pos]

    def step(self, action):
        assert action in [0, 1], action
        if action == 0 and self.cur_pos > 0:
            self.cur_pos -= 1
        elif action == 1:
            self.cur_pos += 1
        done = self.cur_pos >= self.end_pos
        # Produz uma recompensa aleatória quando atingirmos a meta.
        return [self.cur_pos], \
            random.random() * 2 if done else -0.1, done, {}

    def seed(self, seed=None):
        random.seed(seed)

In [18]:
class TorchCustomModel(TorchModelV2, nn.Module):
    """Exemplo de um modelo personalizado PyTorch que apenas delega para uma 
    fc-net."""

    def __init__(self, obs_space, action_space, num_outputs, model_config,
                 name):
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs,
                              model_config, name)
        nn.Module.__init__(self)

        self.torch_sub_model = TorchFC(obs_space, action_space, num_outputs,
                                       model_config, name)

    def forward(self, input_dict, state, seq_lens):
        input_dict["obs"] = input_dict["obs"].float()
        fc_out, _ = self.torch_sub_model(input_dict, state, seq_lens)
        return fc_out, []

    def value_function(self):
        return torch.reshape(self.torch_sub_model.value_function(), [-1])

In [19]:
# Também pode registrar a função de criar um ambiente explicitamente com:
# register_env("corridor", lambda config: SimpleCorridor(config))

# Registrar o modelo customizado
ModelCatalog.register_custom_model(
    "my_model", TorchCustomModel
)

config = {
    "env": SimpleCorridor,  # ou "corridor" se registrado
    "env_config": {
        "corridor_length": 5,
    },
    "model": {
        "custom_model": "my_model",
        "vf_share_layers": True,
    },
    "num_workers": 1,  
    "framework": "torch",
}

stop = {
    "training_iteration": 50,
    "timesteps_total": 100000,
    "episode_reward_mean": 0.1,
}

In [ ]:
pg_config = pg.DEFAULT_CONFIG.copy()
pg_config.update(config)
pg_config["lr"] = 1e-3

trainer = pg.PGTrainer(config=pg_config, env=SimpleCorridor)
# executa o loop de treinamento manual e imprime os resultados após cada iteração
for _ in range(stop["training_iteration"]):
    result = trainer.train()
    print(pretty_print(result))
    
    # pare o treinamento caso tiver alcançado a quantidade de steps desejada
    # ou caso a recompensa desejada seja alcançada
    if result["timesteps_total"] >= stop["timesteps_total"] or \
            result["episode_reward_mean"] >= stop["episode_reward_mean"]:
        break

### Ray Tune

Todos os Trainers do RLlib são compatíveis com a API do [Ray Tune](https://docs.ray.io/en/master/tune/index.html). Isso permite que eles sejam facilmente usados em experimentos com o Tune. Por exemplo, o código a seguir executa o mesmo treino com o CartPole com o algoritmo PG.

In [ ]:
import ray
config = {
    "env": environment_id,
    "framework": "torch",
}
stop = {"episode_reward_mean": 150, "timesteps_total": 100000}

# Executar o treinamento
analysis = ray.tune.run(
    "PG",
    config=config,
    stop=stop,
    checkpoint_freq=10,
    checkpoint_at_end=True,
    local_dir=os.path.join(DRIVE_PATH, "results")
)

Embora o objeto de análise retornado do `ray.tune.run` anteriormente não tivesse nenhuma instância Trainer, ele tem todas as informações necessárias para reconstruir um de um checkpoint salvo.

O retorno do Ray Tune é um objeto [ExperimentAnalysis](https://docs.ray.io/en/latest/tune/api_docs/analysis.html?highlight=ExperimentAnalysis#experimentanalysis-tune-experimentanalysis) onde é possível resgatar qual o melhor checkpoint do treino.

In [22]:
from ray.rllib.agents.pg import PGTrainer

# restaurar um Trainer 
trial = analysis.get_best_logdir("episode_reward_mean", "max")
checkpoint = analysis.get_best_checkpoint(
  trial,
  "training_iteration",
  "max",
)
trainer = PGTrainer(config=config)
trainer.restore(checkpoint)

2021-11-12 23:57:37,585	WARNING util.py:53 -- Install gputil for GPU system monitoring.
2021-11-12 23:57:37,588	INFO trainable.py:378 -- Restored on 192.168.130.2 from checkpoint: /home/eduardo/ceia/curso-rl-ceia-2021/labs/minicurso_rl/lab03/results/PG/PG_CartPole-v0_6ac44_00000_0_2021-11-12_23-57-19/checkpoint_000099/checkpoint-99
2021-11-12 23:57:37,589	INFO trainable.py:385 -- Current state after restoring: {'_iteration': 99, '_timesteps_total': None, '_time_total': 13.992749214172363, '_episodes_total': 219}


Agora vamos criar outro vídeo, mas desta vez escolha a ação recomendada pelo modelo treinado com a API Tune.

In [23]:
after_training = after_training = os.path.join(
    DRIVE_PATH, "{}after_training_tune.mp4".format(environment_id)
)
after_video = VideoRecorder(env, after_training)
observation = env.reset()
done = False
while not done:
  env.render()
  after_video.capture_frame()
  action = trainer.compute_action(observation)
  observation, reward, done, info = env.step(action)
after_video.close()
env.close()
# You should get a video similar to the one below. 
html = render_mp4(after_training)
HTML(html)

O Tune gera arquivos do [Tensorboard](https://www.tensorflow.org/tensorboard) automaticamente durante o `tune.run()` Para visualizar a aprendizagem no tensorboard, execute o célula abaixo:

In [24]:
%tensorboard --logdir ../minicurso_rl/lab03/results/PG

## Hyperparameter Tuning com o Ray Tune

[Ray Tune](https://docs.ray.io/en/latest/tune/index.html) é uma biblioteca para execução de experimentos e ajuste de hiperparâmetros. Vamos agora tentar encontrar hiperparâmetros que possam resolver o ambiente [Cartpole](https://gym.openai.com/envs/CartPole-v1/) no menor número de passos de tempo. Esteja preparado para que demore um pouco para ser executado.

In [ ]:
parameter_search_config = {
    "env": environment_id,
    "framework": "torch",
    "num_gpus": 1,  # porcentagem da gpu disponível para treino
    "num_workers": 2,

    # Hyperparameter tuning
    "model": {
      "fcnet_hiddens": ray.tune.grid_search([[32], [64]]),
      "fcnet_activation": ray.tune.grid_search(["linear", "relu"]),
    },
    "lr": ray.tune.uniform(1e-7, 1e-2)
}

# To explicitly stop or restart Ray, use the shutdown API.
ray.shutdown()

ray.init(
  num_cpus=3,
  include_dashboard=False,
  ignore_reinit_error=True,
  log_to_driver=False,
)

parameter_search_analysis = ray.tune.run(
  "PG",
  config=parameter_search_config,
  stop=stop,
  num_samples=5,
  metric="timesteps_total",
  mode="min",
)

Trial name,status,loc,lr,model/fcnet_activation,model/fcnet_hiddens
PG_CartPole-v0_2b2c7_00000,PENDING,,0.00241435,linear,[32]
PG_CartPole-v0_2b2c7_00001,PENDING,,0.00553195,relu,[32]
PG_CartPole-v0_2b2c7_00002,PENDING,,0.00477552,linear,[64]
PG_CartPole-v0_2b2c7_00003,PENDING,,0.00565651,relu,[64]
PG_CartPole-v0_2b2c7_00004,PENDING,,0.00111003,linear,[32]
PG_CartPole-v0_2b2c7_00005,PENDING,,0.00122893,relu,[32]
PG_CartPole-v0_2b2c7_00006,PENDING,,0.00924176,linear,[64]
PG_CartPole-v0_2b2c7_00007,PENDING,,0.000904234,relu,[64]
PG_CartPole-v0_2b2c7_00008,PENDING,,0.00576621,linear,[32]
PG_CartPole-v0_2b2c7_00009,PENDING,,0.00472464,relu,[32]


Trial name,status,loc,lr,model/fcnet_activation,model/fcnet_hiddens
PG_CartPole-v0_2b2c7_00000,RUNNING,,0.00241435,linear,[32]
PG_CartPole-v0_2b2c7_00001,PENDING,,0.00553195,relu,[32]
PG_CartPole-v0_2b2c7_00002,PENDING,,0.00477552,linear,[64]
PG_CartPole-v0_2b2c7_00003,PENDING,,0.00565651,relu,[64]
PG_CartPole-v0_2b2c7_00004,PENDING,,0.00111003,linear,[32]
PG_CartPole-v0_2b2c7_00005,PENDING,,0.00122893,relu,[32]
PG_CartPole-v0_2b2c7_00006,PENDING,,0.00924176,linear,[64]
PG_CartPole-v0_2b2c7_00007,PENDING,,0.000904234,relu,[64]
PG_CartPole-v0_2b2c7_00008,PENDING,,0.00576621,linear,[32]
PG_CartPole-v0_2b2c7_00009,PENDING,,0.00472464,relu,[32]


Result for PG_CartPole-v0_2b2c7_00000:
  agent_timesteps_total: 400
  custom_metrics:
    default_policy: {}
  date: 2021-11-13_03-44-43
  done: false
  episode_len_mean: 23.666666666666668
  episode_media: {}
  episode_reward_max: 41.0
  episode_reward_mean: 23.666666666666668
  episode_reward_min: 12.0
  episodes_this_iter: 12
  episodes_total: 12
  experiment_id: 40605ae4361e4247bd74c7999a2ca42b
  hostname: eduardo-G7-7588
  info:
    learner:
      default_policy:
        allreduce_latency: 0.0
        policy_loss: 11.35116195678711
    num_agent_steps_sampled: 400
    num_steps_sampled: 400
    num_steps_trained: 400
  iterations_since_restore: 1
  node_ip: 192.168.130.2
  num_healthy_workers: 2
  off_policy_estimator: {}
  perf:
    cpu_util_percent: 29.5
    ram_util_percent: 75.5
  pid: 29315
  policy_reward_max: {}
  policy_reward_mean: {}
  policy_reward_min: {}
  sampler_perf:
    mean_action_processing_ms: 0.03043039521174644
    mean_env_render_ms: 0.0
    mean_env_wait_ms

Trial name,status,loc,lr,model/fcnet_activation,model/fcnet_hiddens,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PG_CartPole-v0_2b2c7_00000,RUNNING,192.168.130.2:29315,0.00241435,linear,[32],31,3.97008,12400,51.75,183,13,51.75
PG_CartPole-v0_2b2c7_00001,PENDING,,0.00553195,relu,[32],,,,,,,
PG_CartPole-v0_2b2c7_00002,PENDING,,0.00477552,linear,[64],,,,,,,
PG_CartPole-v0_2b2c7_00003,PENDING,,0.00565651,relu,[64],,,,,,,
PG_CartPole-v0_2b2c7_00004,PENDING,,0.00111003,linear,[32],,,,,,,
PG_CartPole-v0_2b2c7_00005,PENDING,,0.00122893,relu,[32],,,,,,,
PG_CartPole-v0_2b2c7_00006,PENDING,,0.00924176,linear,[64],,,,,,,
PG_CartPole-v0_2b2c7_00007,PENDING,,0.000904234,relu,[64],,,,,,,
PG_CartPole-v0_2b2c7_00008,PENDING,,0.00576621,linear,[32],,,,,,,
PG_CartPole-v0_2b2c7_00009,PENDING,,0.00472464,relu,[32],,,,,,,


Result for PG_CartPole-v0_2b2c7_00000:
  agent_timesteps_total: 15200
  custom_metrics:
    default_policy: {}
  date: 2021-11-13_03-44-48
  done: false
  episode_len_mean: 59.59
  episode_media: {}
  episode_reward_max: 200.0
  episode_reward_mean: 59.59
  episode_reward_min: 13.0
  episodes_this_iter: 6
  episodes_total: 358
  experiment_id: 40605ae4361e4247bd74c7999a2ca42b
  hostname: eduardo-G7-7588
  info:
    learner:
      default_policy:
        allreduce_latency: 0.0
        policy_loss: 18.762834548950195
    num_agent_steps_sampled: 15200
    num_steps_sampled: 15200
    num_steps_trained: 15200
  iterations_since_restore: 38
  node_ip: 192.168.130.2
  num_healthy_workers: 2
  off_policy_estimator: {}
  perf: {}
  pid: 29315
  policy_reward_max: {}
  policy_reward_mean: {}
  policy_reward_min: {}
  sampler_perf:
    mean_action_processing_ms: 0.029449517173246117
    mean_env_render_ms: 0.0
    mean_env_wait_ms: 0.042415734771822707
    mean_inference_ms: 0.47698346724906443

...

Result for PG_CartPole-v0_2b2c7_00016:
  agent_timesteps_total: 32400
  custom_metrics:
    default_policy: {}
  date: 2021-11-13_03-51-12
  done: true
  episode_len_mean: 151.25
  episode_media: {}
  episode_reward_max: 200.0
  episode_reward_mean: 151.25
  episode_reward_min: 34.0
  episodes_this_iter: 2
  episodes_total: 399
  experiment_id: 9fe901f5e96d48ac948358a200193a42
  hostname: eduardo-G7-7588
  info:
    learner:
      default_policy:
        allreduce_latency: 0.0
        policy_loss: 22.069923400878906
    num_agent_steps_sampled: 32400
    num_steps_sampled: 32400
    num_steps_trained: 32400
  iterations_since_restore: 81
  node_ip: 192.168.130.2
  num_healthy_workers: 2
  off_policy_estimator: {}
  perf: {}
  pid: 31219
  policy_reward_max: {}
  policy_reward_mean: {}
  policy_reward_min: {}
  sampler_perf:
    mean_action_processing_ms: 0.03150099491374513
    mean_env_render_ms: 0.0
    mean_env_wait_ms: 0.046035884499978585
    mean_inference_ms: 0.5071704364137378


Trial name,status,loc,lr,model/fcnet_activation,model/fcnet_hiddens,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PG_CartPole-v0_2b2c7_00017,RUNNING,192.168.130.2:31375,0.00799952,relu,[32],1,0.148624,400,23.3125,64,9,23.3125
PG_CartPole-v0_2b2c7_00018,PENDING,,0.00850983,linear,[64],,,,,,,
PG_CartPole-v0_2b2c7_00019,PENDING,,0.000209516,relu,[64],,,,,,,
PG_CartPole-v0_2b2c7_00000,TERMINATED,,0.00241435,linear,[32],93,12.6749,37200,151.51,200,27,151.51
PG_CartPole-v0_2b2c7_00001,TERMINATED,,0.00553195,relu,[32],88,12.1834,35200,150.38,200,55,150.38
PG_CartPole-v0_2b2c7_00002,TERMINATED,,0.00477552,linear,[64],56,7.91449,22400,150.37,200,44,150.37
PG_CartPole-v0_2b2c7_00003,TERMINATED,,0.00565651,relu,[64],68,9.99143,27200,151.89,200,31,151.89
PG_CartPole-v0_2b2c7_00004,TERMINATED,,0.00111003,linear,[32],202,27.7631,80800,152.08,200,22,152.08
PG_CartPole-v0_2b2c7_00005,TERMINATED,,0.00122893,relu,[32],235,32.9418,94000,150.33,200,29,150.33
PG_CartPole-v0_2b2c7_00006,TERMINATED,,0.00924176,linear,[64],49,6.283,19600,151.22,200,41,151.22


Result for PG_CartPole-v0_2b2c7_00017:
  agent_timesteps_total: 14400
  custom_metrics:
    default_policy: {}
  date: 2021-11-13_03-51-23
  done: false
  episode_len_mean: 73.91
  episode_media: {}
  episode_reward_max: 200.0
  episode_reward_mean: 73.91
  episode_reward_min: 18.0
  episodes_this_iter: 3
  episodes_total: 284
  experiment_id: 40334f42cc63473a960bb0ad8ba121f5
  hostname: eduardo-G7-7588
  info:
    learner:
      default_policy:
        allreduce_latency: 0.0
        policy_loss: 19.081628799438477
    num_agent_steps_sampled: 14400
    num_steps_sampled: 14400
    num_steps_trained: 14400
  iterations_since_restore: 36
  node_ip: 192.168.130.2
  num_healthy_workers: 2
  off_policy_estimator: {}
  perf: {}
  pid: 31375
  policy_reward_max: {}
  policy_reward_mean: {}
  policy_reward_min: {}
  sampler_perf:
    mean_action_processing_ms: 0.03144545006250001
    mean_env_render_ms: 0.0
    mean_env_wait_ms: 0.045233969583375035
    mean_inference_ms: 0.517446284752933
  

Trial name,status,loc,lr,model/fcnet_activation,model/fcnet_hiddens,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PG_CartPole-v0_2b2c7_00017,RUNNING,192.168.130.2:31375,0.00799952,relu,[32],36,4.89211,14400,73.91,200,18,73.91
PG_CartPole-v0_2b2c7_00018,PENDING,,0.00850983,linear,[64],,,,,,,
PG_CartPole-v0_2b2c7_00019,PENDING,,0.000209516,relu,[64],,,,,,,
PG_CartPole-v0_2b2c7_00000,TERMINATED,,0.00241435,linear,[32],93,12.6749,37200,151.51,200,27,151.51
PG_CartPole-v0_2b2c7_00001,TERMINATED,,0.00553195,relu,[32],88,12.1834,35200,150.38,200,55,150.38
PG_CartPole-v0_2b2c7_00002,TERMINATED,,0.00477552,linear,[64],56,7.91449,22400,150.37,200,44,150.37
PG_CartPole-v0_2b2c7_00003,TERMINATED,,0.00565651,relu,[64],68,9.99143,27200,151.89,200,31,151.89
PG_CartPole-v0_2b2c7_00004,TERMINATED,,0.00111003,linear,[32],202,27.7631,80800,152.08,200,22,152.08
PG_CartPole-v0_2b2c7_00005,TERMINATED,,0.00122893,relu,[32],235,32.9418,94000,150.33,200,29,150.33
PG_CartPole-v0_2b2c7_00006,TERMINATED,,0.00924176,linear,[64],49,6.283,19600,151.22,200,41,151.22


Result for PG_CartPole-v0_2b2c7_00017:
  agent_timesteps_total: 27600
  custom_metrics:
    default_policy: {}
  date: 2021-11-13_03-51-28
  done: true
  episode_len_mean: 150.56
  episode_media: {}
  episode_reward_max: 200.0
  episode_reward_mean: 150.56
  episode_reward_min: 53.0
  episodes_this_iter: 3
  episodes_total: 365
  experiment_id: 40334f42cc63473a960bb0ad8ba121f5
  hostname: eduardo-G7-7588
  info:
    learner:
      default_policy:
        allreduce_latency: 0.0
        policy_loss: 18.816303253173828
    num_agent_steps_sampled: 27600
    num_steps_sampled: 27600
    num_steps_trained: 27600
  iterations_since_restore: 69
  node_ip: 192.168.130.2
  num_healthy_workers: 2
  off_policy_estimator: {}
  perf:
    cpu_util_percent: 30.6
    ram_util_percent: 75.9
  pid: 31375
  policy_reward_max: {}
  policy_reward_mean: {}
  policy_reward_min: {}
  sampler_perf:
    mean_action_processing_ms: 0.0309108381472501
    mean_env_render_ms: 0.0
    mean_env_wait_ms: 0.04413659650

In [ ]:
print(
  "Melhores hiperparâmetros encontrados:",
  parameter_search_analysis.best_config,
)

Especificando num_samples = 5 significa que você obterá cinco amostras aleatórias para a taxa de aprendizagem. Para cada um deles, existem dois valores para o tamanho da camada oculta e dois valores para a função de ativação. Portanto, haverá 5 * 2 * 2 = 20 tentativas, mostradas com seus status na saída da célula à medida que o cálculo é executado.

Observe que Ray mostra a melhor configuração atual à medida que avança. Isso inclui todos os valores padrão que foram definidos, o que é um bom lugar para encontrar outros parâmetros que podem ser ajustados.


## Exercício

Agora que você conhece a API básica do Ray Tune e da RLLib, **utilize o ambiente `BreakoutNoFrameskip-v4` e treine agentes com os algoritmos A3C, PPO e SAC**. Lembre-se de utilizar também o tensorboard para acompanhar e comparar as curvas de aprendizado de suas execuções.

Descrições dos algoritmos e seus respectivos hiperparâmetros podem ser encontrados [aqui](https://docs.ray.io/en/latest/rllib-algorithms.html#available-algorithms-overview).

In [39]:
#reimport libs
import ray
import os

environment_id = 'BreakoutNoFrameskip-v4'

# To explicitly stop or restart Ray, use the shutdown API.
ray.shutdown()

ray.init(
  include_dashboard=False,
  ignore_reinit_error=True,
  log_to_driver=False,
)

{'node_ip_address': '192.168.130.2',
 'raylet_ip_address': '192.168.130.2',
 'redis_address': '192.168.130.2:6379',
 'object_store_address': '/tmp/ray/session_2021-11-13_00-45-44_824627_22643/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-11-13_00-45-44_824627_22643/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2021-11-13_00-45-44_824627_22643',
 'metrics_export_port': 43323,
 'node_id': 'd8b5c6d4e72a8ccbfaff032747917213ae1b88dbc814c70d613328dd'}

In [74]:
def query_environment(name):
    
  env = gym.make(name)
  spec = gym.spec(name)

  print(f"Action Space: {env.action_space}")
  print(f"Observation Space: {env.observation_space}")
  print(f"Max Episode Steps: {spec.max_episode_steps}")
  print(f"Nondeterministic: {spec.nondeterministic}")
  print(f"Reward Range: {env.reward_range}")
  print(f"Reward Threshold: {spec.reward_threshold}")

In [34]:
query_environment(environment_id)

Action Space: Discrete(4)
Observation Space: Box([[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]], [[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255

In [ ]:
a3c_analysis = ray.tune.run(
    "A3C",
    config={
        # RL setup
        "env": environment_id,
        # system settings
        "num_gpus": 1,
        "num_workers": 2,
        "num_envs_per_worker": NUM_ENVS_PER_WORKER,
        "log_level": "INFO",
        "framework": "torch",    
    },
    stop={
        # você também pode limitar por tempo, de acordo com o tempo limite do colab
        "time_total_s": 1200, # 20min
    },
    checkpoint_freq=50,
    checkpoint_at_end=True,
    local_dir=os.path.join("../minicurso_rl/lab03/results", "breakout")
)

In [ ]:
trial = a3c_analysis.get_best_trial("episode_reward_mean", "max")
checkpoint = a3c_analysis.get_best_checkpoint(
  trial,
  "episode_reward_mean",
  "max",
)
print('A3C Results (20min):')
print('episode_reward_max', trial.last_result['episode_reward_max'])
print('episode_reward_min', trial.last_result['episode_reward_min'])
print('episode_reward_mean', trial.last_result['episode_reward_mean'])
print('episode_len_mean', trial.last_result['episode_len_mean'])
print('checkpoint', checkpoint)

In [ ]:
ppo_analysis = ray.tune.run(
    "PPO",
    config={
        # RL setup
        "env": environment_id,
        # system settings
        "num_gpus": 1,
        "num_workers": 2,
        "num_envs_per_worker": 5,
        "log_level": "INFO",
        "framework": "torch",
        
        # Hiperparametros obtidos de:
        # https://github.com/ray-project/ray/blob/master/rllib/tuned_examples/ppo/atari-ppo.yaml
        "lambda": 0.95,
        "kl_coeff": 0.5,
        "clip_rewards": True,
        "clip_param": 0.1,
        "vf_clip_param": 10.0,
        "entropy_coeff": 0.01,
        "train_batch_size": 5000,
        "rollout_fragment_length": 100,
        "sgd_minibatch_size": 500,
        "num_sgd_iter": 10,
        "batch_mode": "truncate_episodes",
        "observation_filter": "NoFilter",
        "model": { "vf_share_layers": True }
    },
    stop={
        # você também pode limitar por tempo, de acordo com o tempo limite do colab
        "time_total_s": 1200, # 20min
    },
    checkpoint_freq=100,
    checkpoint_at_end=True,
    local_dir=os.path.join("../minicurso_rl/lab03", "results/breakout")
)

In [ ]:
trial = ppo_analysis.get_best_trial("episode_reward_mean", "max")
checkpoint = ppo_analysis.get_best_checkpoint(
  trial,
  "episode_reward_mean",
  "max",
)
print('PPO Results (20min):')
print('episode_reward_max', trial.last_result['episode_reward_max'])
print('episode_reward_min', trial.last_result['episode_reward_min'])
print('episode_reward_mean', trial.last_result['episode_reward_mean'])
print('episode_len_mean', trial.last_result['episode_len_mean'])
print('checkpoint', checkpoint)

In [ ]:
sac_analysis = ray.tune.run(
    "SAC",
    config={
        # RL setup
        "env": environment_id,
        # system settings
        "num_gpus": 1,
        "num_workers": 0,
        "log_level": "INFO",
        "framework": "torch",
        
        #limitar replay buffer size (padrão estorou memoria)
        "buffer_size": int(1e5),
        
        # Artigo usa 20k timesteps para começar o treinamento,
        # O tunning do ray usa 100k
        # mas por limites de memoria e o buffer ser pequeno vamos com 5k
        "learning_starts": 5000,
        
        # Hiperparametros obtidos de: (modificado) (outros env's, não otimizado para o BreakoutNoFrameskip-v4)
        #https://github.com/ray-project/ray/blob/master/rllib/tuned_examples/sac/atari-sac.yaml
        "gamma": 0.99,
        "Q_model": {
                "fcnet_activation": "relu",
                "fcnet_hiddens": [512, 512]
            },
        "policy_model" :{
                "fcnet_activation": "relu",
                "fcnet_hiddens": [512, 512]
            },
        # Do hard syncs.
        # Soft-syncs seem to work less reliably for discrete action spaces.
        "tau": 1.0,
        "target_network_update_freq": 8000,
        "target_entropy": "auto",
        "clip_rewards": 1.0,
        "n_step": 1,
        "rollout_fragment_length": 1,
        "prioritized_replay": True,
        "train_batch_size": 64,
        "timesteps_per_iteration": 4,
        "optimization": {
                "actor_learning_rate": 0.0003,
                "critic_learning_rate": 0.0003,
                "entropy_learning_rate": 0.0003,
            },
        "metrics_smoothing_episodes": 5
    },
    stop={
        # você também pode limitar por tempo, de acordo com o tempo limite do colab
        "time_total_s": 1200, # 20min
    },
    checkpoint_freq=50,
    checkpoint_at_end=True,
    local_dir=os.path.join("../minicurso_rl/lab03", "results/breakout")
)

In [79]:
trial = sac_analysis.get_best_trial("episode_reward_mean", "max")
checkpoint = sac_analysis.get_best_checkpoint(
  trial,
  "episode_reward_mean",
  "max",
)
print('SAC Results (20min):')
print('episode_reward_max', trial.last_result['episode_reward_max'])
print('episode_reward_min', trial.last_result['episode_reward_min'])
print('episode_reward_mean', trial.last_result['episode_reward_mean'])
print('episode_len_mean', trial.last_result['episode_len_mean'])
print('checkpoint', checkpoint)

SAC Results:
episode_reward_max 9.0
episode_reward_min 0.0
episode_reward_mean 3.6
episode_len_mean 974.0
checkpoint /home/eduardo/ceia/curso-rl-ceia-2021/labs/minicurso_rl/lab03/results/breakout/SAC/SAC_BreakoutNoFrameskip-v4_dd6e1_00000_0_2021-11-13_02-45-10/checkpoint_000750/checkpoint-750


In [87]:
%tensorboard --logdir ../minicurso_rl/lab03/results/breakout

Reusing TensorBoard on port 6006 (pid 10664), started 2:41:32 ago. (Use '!kill 10664' to kill it.)

# Bônus

Como tarefa bônus, experimente com os algoritmos aprendidos no ambiente `soccer_twos`, que será utilizado na competição final deste curso*. Para facilitar, utilize a variação `team_vs_policy` como no laboratório anterior.

<img src="https://raw.githubusercontent.com/bryanoliveira/soccer-twos-env/master/images/screenshot.png" height="400">

> Visualização do ambiente

Este ambiente consiste em um jogo de futebol de carros 2x2, ou seja, o objetivo é marcar um gol no adversário o mais rápido possível. Na variação `team_vs_policy`, seu agente controla um jogador do time azul e joga contra um time aleatório. Mais informações sobre o ambiente podem ser encontradas [no repositório](https://github.com/bryanoliveira/soccer-twos-env) e [na documentação do Unity ml-agents](https://github.com/Unity-Technologies/ml-agents/blob/main/docs/Learning-Environment-Examples.md#soccer-twos).


**Sua tarefa é treinar um agente com a interface do Ray apresentada, experimentando com diferentes algoritmos e hiperparâmetros.**


<br>

*A variação utilizada na competição será a `multiagent_player`, mas agentes treinados para `team_vs_policy` podem ser facilmente adaptados. Na seção "Exportando seu agente treinado" o agente "MyDqnSoccerAgent" faz exatamente isso.

Utilize o ambiente instanciado abaixo para executar o algoritmo de treinamento. Ao final da execução, a recompensa do seu agente por episódio deve tender a +2.

In [ ]:
import soccer_twos

# Fecha o ambiente caso tenha sido aberto anteriormente
try: env.close()
except: pass

env = soccer_twos.make(
    variation=soccer_twos.EnvType.team_vs_policy,
    flatten_branched=True, # converte o action_space de MultiDiscrete para Discrete
    single_player=True, # controla um dos jogadores enquanto os outros ficam parados
    opponent_policy=lambda *_: 0,  # faz os oponentes ficarem parados
)

# Obtem tamanhos de estado e ação
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

print("Tamanho do estado: {}, tamanho da ação: {}".format(state_size, action_size))
env.close()

In [ ]:
from ray import tune

def create_rllib_env(env_config: dict = {}):
    # suporte a múltiplas instâncias do ambiente na mesma máquina
    if hasattr(env_config, "worker_index"):
        env_config["worker_id"] = (
            env_config.worker_index * env_config.get("num_envs_per_worker", 1)
            + env_config.vector_index
        )
    return soccer_twos.make(**env_config)

# registra ambiente no Ray
tune.registry.register_env("Soccer", create_rllib_env)

Utilize a configuração abaixo como ponto de partida para seus testes. 

A parte mais imporante é a chave `env_config`, que configura o ambiente para ser compatível com o agente disponibilizado para exportação do seu agente. Neste ponto do curso você já deve conseguir testar as outras variações do ambiente e utilizar as APIs do Ray para treinar um agente próximo (ou melhor) do que o [ceia_baseline_agent](https://drive.google.com/file/d/1WEjr48D7QG9uVy1tf4GJAZTpimHtINzE/view). Exemplos de como utilizar as outras variações podem ser encontrados [aqui](https://github.com/dlb-rl/rl-tournament-starter/). Ao utilizar essas variações, você deve utilizar também outras definições de agente para lidar com os diferentes espaços de observação e ação (que também estão presentes nos exemplos).

In [ ]:
NUM_ENVS_PER_WORKER = 2

analysis = tune.run(
    "PPO",
    config={
        # system settings
        "num_gpus": 1,
        "num_workers": 1,
        "num_envs_per_worker": NUM_ENVS_PER_WORKER,
        "log_level": "INFO",
        "framework": "torch",
        # RL setup
        "env": "Soccer",
        "env_config": {
            "num_envs_per_worker": NUM_ENVS_PER_WORKER,
            "variation": soccer_twos.EnvType.team_vs_policy,
            "single_player": True,
            "flatten_branched": True,
            "opponent_policy": lambda *_: 0,
        },
    },
    stop={
        # 10000000 (10M) de steps podem ser necessários para aprender uma política útil
        "timesteps_total": 10000000,
        # você também pode limitar por tempo, de acordo com o tempo limite do colab
        "time_total_s": 14400, # 4h
    },
    checkpoint_freq=100,
    checkpoint_at_end=True,
    local_dir=os.path.join(DRIVE_PATH, "results")
)

## Exportando seu agente treinado

Assim como no Lab 02, você pode exportar seu agente treinado para ser executado como competidor no ambiente da competição ou simplesmente assistí-lo. Para isso, devemos definir uma classe de agente que implemente a interface e trate as observações/ações para o formato da competição. Abaixo, configuramos qual experimento/checkpoint exportar e guardamos a implementação em uma variável para salvá-la em um arquivo posteriormente.

In [ ]:
ALGORITHM = "PPO"
TRIAL = analysis.get_best_logdir("episode_reward_mean", "max")
CHECKPOINT = analysis.get_best_checkpoint(
  TRIAL,
  "training_iteration",
  "max",
)
TRIAL, CHECKPOINT

In [ ]:
agent_file = f"""
import pickle
import os

import gym
from gym_unity.envs import ActionFlattener
import ray
from ray import tune
from ray.tune.registry import get_trainable_cls

from soccer_twos import AgentInterface, DummyEnv


ALGORITHM = "{ALGORITHM}"
CHECKPOINT_PATH = os.path.join(
    os.path.dirname(os.path.abspath(__file__)), 
    "{CHECKPOINT.split("lab03/")[1]}"
)


class MyRaySoccerAgent(AgentInterface):
    def __init__(self, env: gym.Env):
        super().__init__()
        ray.init(ignore_reinit_error=True)

        self.flattener = ActionFlattener(env.action_space.nvec)

        # Load configuration from checkpoint file.
        config_path = ""
        if CHECKPOINT_PATH:
            config_dir = os.path.dirname(CHECKPOINT_PATH)
            config_path = os.path.join(config_dir, "params.pkl")
            # Try parent directory.
            if not os.path.exists(config_path):
                config_path = os.path.join(config_dir, "../params.pkl")

        # Load the config from pickled.
        if os.path.exists(config_path):
            with open(config_path, "rb") as f:
                config = pickle.load(f)
        else:
            # If no config in given checkpoint -> Error.
            raise ValueError(
                "Could not find params.pkl in either the checkpoint dir or "
                "its parent directory!"
            )

        # no need for parallelism on evaluation
        config["num_workers"] = 0
        config["num_gpus"] = 0

        # create a dummy env since it's required but we only care about the policy
        obs_space = env.observation_space
        act_space = self.flattener.action_space
        tune.registry.register_env(
            "DummyEnv",
            lambda *_: DummyEnv(obs_space, act_space),
        )
        config["env"] = "DummyEnv"

        # create the Trainer from config
        cls = get_trainable_cls(ALGORITHM)
        agent = cls(env=config["env"], config=config)
        # load state from checkpoint
        agent.restore(CHECKPOINT_PATH)
        # get default policy for evaluation
        self.policy = agent.get_policy()

    def act(self, observation):
        actions = {{}}
        for player_id in observation:
            # compute_single_action returns a tuple of (action, action_info, ...)
            # as we only need the action, we discard the other elements
            actions[player_id] = self.flattener.lookup_action(
                self.policy.compute_single_action(observation[player_id])[0]
            )
        return actions
"""

In [ ]:
import os
import shutil

agent_name = "my_ray_soccer_agent"
agent_path = os.path.join(DRIVE_PATH, agent_name, agent_name)
shutil.rmtree(agent_path)
os.makedirs(agent_path)

# salva a classe do agente
with open(os.path.join(agent_path, "agent.py"), "w") as f:
    f.write(agent_file)

# salva um __init__ para criar o módulo Python
with open(os.path.join(agent_path, "__init__.py"), "w") as f:
    f.write("from .agent import MyRaySoccerAgent")

# copia o trial inteiro, incluindo os arquivos de configuração do experimento
shutil.copytree(TRIAL, os.path.join(agent_path, TRIAL.split("lab03/")[1]))

# empacota tudo num arquivo .zip
shutil.make_archive(os.path.join(DRIVE_PATH, agent_name), "zip", os.path.join(DRIVE_PATH, agent_name))

Após empacotar todos os arquivos necessários para a execução do seu agente, será criado um arquivo `minicurso_rl/lab03/my_ray_soccer_agent.zip` nos arquivos do Colab e na pasta correspondente no Google Drive. Baixe o arquivo e extraia-o para alguma pasta no seu computador. 

Assumindo que o ambiente Python já está configurado (e.g. os pacotes no [requirements.txt](https://github.com/dlb-rl/rl-tournament-starter/blob/main/requirements.txt) estão instalados), rode `python -m soccer_twos.watch -m my_ray_soccer_agent` para assistir seu agente jogando contra si mesmo. 

Você também pode testar dois agentes diferentes jogando um contra o outro. Utilize o seguinte comando: `python -m soccer_twos.watch -m1 my_ray_soccer_agent -m2 ceia_baseline_agent`. Você pode baixar o agente *ceia_baseline_agent* [aqui](https://drive.google.com/file/d/1WEjr48D7QG9uVy1tf4GJAZTpimHtINzE/view).